# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

In [1]:
import random

In [2]:
#From Quiz 2:

suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades'] 
values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'Jack', 'Queen', 'King', 'Ace']

def make_deck():
    deck = [(suit, value) for suit in suits for value in values]
    return deck

# Testing the function
deck = make_deck()
print(len(deck))  # Should print 52


52


In [3]:
deck #Show all cards

[('Clubs', 2),
 ('Clubs', 3),
 ('Clubs', 4),
 ('Clubs', 5),
 ('Clubs', 6),
 ('Clubs', 7),
 ('Clubs', 8),
 ('Clubs', 9),
 ('Clubs', 10),
 ('Clubs', 'Jack'),
 ('Clubs', 'Queen'),
 ('Clubs', 'King'),
 ('Clubs', 'Ace'),
 ('Diamonds', 2),
 ('Diamonds', 3),
 ('Diamonds', 4),
 ('Diamonds', 5),
 ('Diamonds', 6),
 ('Diamonds', 7),
 ('Diamonds', 8),
 ('Diamonds', 9),
 ('Diamonds', 10),
 ('Diamonds', 'Jack'),
 ('Diamonds', 'Queen'),
 ('Diamonds', 'King'),
 ('Diamonds', 'Ace'),
 ('Hearts', 2),
 ('Hearts', 3),
 ('Hearts', 4),
 ('Hearts', 5),
 ('Hearts', 6),
 ('Hearts', 7),
 ('Hearts', 8),
 ('Hearts', 9),
 ('Hearts', 10),
 ('Hearts', 'Jack'),
 ('Hearts', 'Queen'),
 ('Hearts', 'King'),
 ('Hearts', 'Ace'),
 ('Spades', 2),
 ('Spades', 3),
 ('Spades', 4),
 ('Spades', 5),
 ('Spades', 6),
 ('Spades', 7),
 ('Spades', 8),
 ('Spades', 9),
 ('Spades', 10),
 ('Spades', 'Jack'),
 ('Spades', 'Queen'),
 ('Spades', 'King'),
 ('Spades', 'Ace')]

In [19]:
#Defining levels of output (logging)
#This will be the base for Card class

class base:
    SILENT=6
    DEBUG=1
    INFO=2
    WARNING=3
    ERROR=4
    CRITICAL=5

    def __init__(self, level=0):
        self.level=level
    
    def message (self, level, *args):
        #Send message if level is higher than current level
        if level >=self.level: 
            print(*args)
    

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [54]:
class Card(base):
    
    __suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades'] 
    __values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'Jack', 'Queen', 'King', 'Ace']

    def __init__(self, suit, value=None):
        base.__init__(self)
        
        #Shuffle card handling
        if suit == "ShuffleCard":
            self.__suit= "ShuffleCard"
            self.__value= None
        
        else:
            #Make sure suits and values are in their list
            self.__suit = suit if suit in self.__suits else None
            self.__value = value if value in self.__values else None
    
            #Error message if suit or value is not in their list
            if self.__suit is None:
                self.message(self.ERROR, "Error, invalid suit:", suit)
    
            if self.__value is None:
                self.message(self.ERROR, "Error, invalid value:", value)

    #Accessors 
    def suit(self):
        return self.__suit
        
    def value(self):
        return self.__value

    def numerical_value(self):
        #Aces can be 1 or 11, need special handling
        if self.__value == 'Ace':
            return 1

        #J, Q, K are 10
        elif self.__value in ['Jack', 'Queen', 'King']:
            return 10

        else:
            return self.__value

    #Make sure its a shuffle card
    def shuffle_card(self):
        return self.__suit =="ShuffleCard"

    #Correctly return str with name of card
    def __str__(self):
        if self.shuffle_card():
            return "Shuffle Card"

        else:
            return str(self.__value) + "  of  " + self.__suit
        
    __repr__ = __str__
        

In [42]:
card0=Card("ShuffleCard")
card0.shuffle_card()

True

In [59]:
card0=Card("Clubs",3)
print(card0.suit())
print(card0.value())
print(card0.numerical_value())

card0=Card("Diamonds",5)
print(card0.suit())
print(card0.value())
print(card0.numerical_value())

Clubs
3
3
Diamonds
5
5


In [66]:
class Deck(base):
    __suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades'] 
    __values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'Jack', 'Queen', 'King', 'Ace']


    #Initialize deck class with 6 decks by default
    def __init__(self,n_decks=6):
        base.__init__(self)
        self.__n_decks=n_decks

        #List of cards in deck, should hold more than 52
        self.__cards=list()

        #Add another 52 cards at the end of our deck each iteration
        for _ in range(self.__n_decks):
            self.__cards.extend(self.__make_deck())


        #Insert shuffle card 
        shuff_card_index=random.randint(234,265) #75%-85% penetration for 6 decks
        self.__cards.insert(shuff_card_index,Card("ShuffleCard"))

   
    #Creates a deck using Card instances
    def __make_deck(self):
        deck=list()
        for suit in self.__suits:
            for value in self.__values:
                deck.append(Card(suit,value))
        return deck
        
    #Method to randomly shuffle deck(s) of cards
    def shuffle(self):
        random.shuffle(self.__cards)

    def deal(self):
        #If there are still cards in the deck, pop a card
        if len(self.__cards)>0:
            pop_card=self.__cards.pop()

            #If shuffle card, shuffle decks and deal again
            if pop_card.shuffle_card():
                self.__cards.shuffle()
                return self.deal()

            return pop_card
            
        #If there are no remaining cards, create new decks and pop a card
        else:
            for _ in range(self.__n_decks):
                self.__cards.extend(self.__make_deck())
            self.shuffle()
            return self.__cards.pop()

     

In [67]:
deck0=Deck()
deck0.shuffle()

[deck0.deal() for _ in range(10)]

[3  of  Diamonds,
 3  of  Diamonds,
 King  of  Spades,
 King  of  Clubs,
 4  of  Diamonds,
 Ace  of  Hearts,
 Jack  of  Diamonds,
 6  of  Hearts,
 King  of  Clubs,
 Jack  of  Hearts]

In [ ]:
#Calculate value of hand and handle aces (can be 1 or 11)
def calc_hand_value(hand):
    
    #List of numerical card values in hand
    card_values = list(map(lambda card: card.numerical_value(),hand))

    #Get number of aces in card_values
    n_As= len(list(filter(lambda x: x==1,card_values)))
    
    hand_value = sum(card_values)

    #If no aces, return hand_value
    if n_As==0:
        return hand_value
    
    Ace_as_one = hand_value 
    Ace_as_eleven = hand_value+10

    #Make sure to have best hand and not bust
    if Ace_as_eleven<=21:
        return Ace_as_eleven
    else:
        return Ace_as_one

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

In [ ]:
class Card(base):

class Deck(base):

class Player(base):
    def __init__(self,name, n_chips):
        base.__init__(self)
        self.__name=name
        self.__n_chips=n_chips

    def name(self):
        return self.__name

    def chips(self):
        return self.__n_chips

    def pay(self):
        return self.__n_chips+=value

    def deduct(self):
        return self.__n_chips-=value

        

    def __str__(self):
        return self.__name + "("+ str(self.__n_chips) + ")"
    
    __repr__=__str__
        




class Game(base):
    def __init__(self, n_decks=6)
        base.__init__(self, self.INFO)

    def calc_hand
        
    
    


4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 